# Audio-Dateien Transkribieren

Mit diesem Code kann man eine Audio-Datei transkribieren:

- Schritt I): Modell wählen. Je größer das Modell, umso genauer die Transkription. Allerdings kann es sein, dass bei größeren Modellen die Rechenkapazitäten nicht mehr ausreichen.
- Schritt II): Datei auswählen. Standardmäßig werden MP3-Dateien vorgeschlagen. Man kann bei der Dateiauswahl rechts unten weitere Datei-Endungen angeben.

Die Word-Datei wird im gleichen Ordner unter dem gleichen Dateinamen wie die ursprüngliche Audio-Datei abgespeichert.

Die Dateigröße darf höchstens 25MB betragen.

<div class="alert alert-block alert-info"> <b>Hinweis:</b> Dieses Notebook transkribiert nur Deutsch. </div>

In [1]:
from pathlib import Path
import FreeSimpleGUI as sg
from docx import Document
import whisper
from pytictoc import TicToc
import warnings
import sys
# user input
path_start = Path(r'.')
my_language = 'de'
my_model = 'large'
warnings.simplefilter(action='ignore', category=Warning)

# Auswahl Modell, Window 1
layout = [[sg.T(""), sg.Radio('tiny (39 M)', "RADIO1", default=False, key="tiny")],
          [sg.T(""), sg.Radio('base (74 M)', "RADIO1", default=False, key="base")],
          [sg.T(""), sg.Radio('small (244 M)', "RADIO1", default=False, key="small")],
          [sg.T(""), sg.Radio('medium (769 M)', "RADIO1", default=False, key="medium")],
          [sg.T(""), sg.Radio('large (1550 M)', "RADIO1", default=True, key="large")],
          [sg.Button('OK'), sg.Button('Abbruch')]
          ]
window = sg.Window('Auswahl Modell', layout, size=(300, 200))
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == "Abbruch":
        print('Abbruch durch Benutzer, Vorgang wiederholen.')
        break
    elif event == 'OK':
        break
window.close()

# Modell laden, Datei auswählen
if event == 'OK':
    # Modell aus Eingabemaske abfangen
    for v in values:
        if values[v]:
            my_model = v
    print('Lade Modell:', my_model)
    try:
        whisper_model = whisper.load_model(my_model)
    except:
        print('Unbekannter Fehler.')

    if 'whisper_model' in globals():
        print('erfolgreich')
        # Datei auswählen
        layout = [[sg.Input(key='_FILE_'),
                   sg.FileBrowse(file_types=(("MP3", ".mp3"), ("*", ".*")), button_text="Auswahl", initial_folder=path_start)],
                  [sg.Button('OK'), sg.Button('Abbruch')]]
        window = sg.Window('Auswahl Audio-Datei', layout)
        event2, values2 = window.read()
        window.close()
    
        if event2 == 'OK':
            t = TicToc()
            t.tic()
            print('Starte Transkription')
            try:
                result = whisper_model.transcribe(values2['_FILE_'], language=my_language, fp16=False)
                print('erfolgreich')
                t.toc()
                # save file
                print('Word-Datei speichern')
                document = Document()
                document.add_paragraph(result["text"])
                try:
                    document.save(str(Path(values2['_FILE_']).parent / Path(values2['_FILE_']).stem) + '.docx')
                    print('erfolgreich')
                    print('Transkription beendet.')
                except PermissionError:
                    print(f'Datei speichern fehlgeschlagen. Möglicherweise ist die Datei  {Path(values2["_FILE_"]).stem}.docx  noch geöffnet.')
    
            except RuntimeError:
                print('Datei einlesen fehlgeschlagen, Dateiformat überprüfen.')
    
        else:
            print('Keine Datei ausgewählt, Vorgang wiederholen.')

Lade Modell: tiny
erfolgreich
Starte Transkription
erfolgreich
Elapsed time is 11.417688 seconds.
Word-Datei speichern
erfolgreich
Transkription beendet.
